
##### Read Excel File "Sales" sheet from ADLS Gen 2

In [2]:
import com.microsoft.spark.fabric
from pyspark.sql.functions import col, round, regexp_replace, trim, when, year, month, quarter, weekofyear, to_date, expr, row_number, lit
from pyspark.sql.types import *
from datetime import datetime, timedelta
from pyspark.sql.window import Window
import pandas as pd

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 4, Finished, Available, Finished)

In [3]:
# Define the path to the Excel file inside Lakehouse
src_path = '/lakehouse/default/Files/project4/Day1/Sales_01012023.xlsx'

# Read the entire "Sales" sheet (all columns)
df = pd.read_excel(src_path, sheet_name='Sales')

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Display the Spark DataFrame
display(spark_df.limit(5))

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 875455df-03d8-4e35-9056-77ed044553c2)

##### Transformation Steps on "Sales"

In [4]:
# Remove duplicates
df_dedup = spark_df.dropDuplicates()

# Cast to appropriate data types
df_casted = df_dedup \
    .withColumn("Order_Date", col("Order_Date").cast(DateType())) \
    .withColumn("Shipping_Date", col("Shipping_Date").cast(DateType())) \
    .withColumn("Aging", col("Aging").cast(IntegerType())) \
    .withColumn("Sales", col("Sales").cast(DoubleType())) \
    .withColumn("Quantity", col("Quantity").cast(IntegerType())) \
    .withColumn("Discount", col("Discount").cast(DoubleType())) \
    .withColumn("Profit", col("Profit").cast(DoubleType())) \
    .withColumn("Shipping_Cost", col("Shipping_Cost").cast(DoubleType()))

# Filter out rows where critical columns are null
df_clean = df_casted.filter(
    col("Order_ID").isNotNull() &
    col("Order_Date").isNotNull() &
    col("Sales").isNotNull() &
    col("Customer_ID").isNotNull()
)

df_clean_rounded = df_clean \
    .withColumn("Shipping_Cost", round(col("Shipping_Cost"), 2)) \
    .withColumn("Sales", round(col("Sales"), 2)) \
    .withColumn("Profit", round(col("Profit"), 2)) \
    .withColumn("Discount", round(col("Discount"), 2))

# Display the cleaned DataFrame
display(df_clean_rounded.limit(5))


StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f500ff5a-7516-4c51-8edb-38c2b713709d)

##### Read Excel File "Returns" sheet from ADLS Gen 2

In [5]:
# Define the path to the Excel file inside Lakehouse
src_path = '/lakehouse/default/Files/project4/Day1/Sales_01012023.xlsx'

# Read the entire "Returns" sheet (all columns)
df = pd.read_excel(src_path, sheet_name='Returns')

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Display the Spark DataFrame
display(spark_df.limit(5))

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3f2abee-156e-43b3-9d61-b321bfa825f2)

#####  Transformation Steps on "Returns"

In [6]:
# Remove duplicates (if any)
df_return_dedup = spark_df.dropDuplicates()

# Clean up and convert 'Sales_Amount' from string ($250.00) to float
df_return_cleaned = df_return_dedup \
    .withColumn("Sales_Amount", regexp_replace(col("Sales_Amount"), "[$]", "").cast("double")) \
    .withColumn("Order_ID", trim(col("Order_ID"))) \
    .withColumn("Customer_Name", trim(col("Customer_Name"))) \
    .withColumn("Return", trim(col("Return")))

# Filter rows where key fields are not null
df_return_final = df_return_cleaned.filter(
    col("Order_ID").isNotNull() & col("Customer_Name").isNotNull() & col("Sales_Amount").isNotNull()
)

# Display result
display(df_return_final.limit(5))

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 323a0c45-507a-41f5-8c5c-6ab845cf54fb)

##### Create Dimension Tables from "Sales"

In [7]:
# ------------------------------------------Dimension - Date Table----------------------------------------------------

# Generate a 10-year date range
start_date = datetime(2020, 1, 1)
end_date = datetime(2030, 12, 31)
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Convert to Spark DataFrame with a single column
df_date = spark.createDataFrame([(d,) for d in date_list], ["BusinessDate"])

# Add business calendar columns
df_date = df_date \
    .withColumn("Business_Year", year("BusinessDate")) \
    .withColumn("Business_Month", month("BusinessDate")) \
    .withColumn("Business_Quarter", quarter("BusinessDate")) \
    .withColumn("Business_Week", weekofyear("BusinessDate"))

# Add Date_ID as surrogate key
df_date_with_id = df_date.withColumn(
    "Date_ID", row_number().over(Window.orderBy("BusinessDate"))
)

# Reorder columns
df_date_with_id = df_date_with_id.select(
    "Date_ID", "BusinessDate", "Business_Year", "Business_Month", "Business_Quarter", "Business_Week"
)

# Write to Warehouse as table
df_date_with_id.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_date')

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 9, Finished, Available, Finished)

In [8]:
# -------------------------------------------Dimension - Segment-----------------------------------------------------
df_segment = df_clean_rounded.select("Segment").dropDuplicates()

df_segment_dim = df_segment.withColumn(
    "Segment_ID", row_number().over(Window.orderBy("Segment"))
)

df_segment_dim = df_segment_dim.select("Segment_ID", "Segment")

# Write to Warehouse as table
df_segment_dim.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_segment')

# ------------------------------------------Dimension - Customer----------------------------------------------------
dim_customer = df_clean_rounded.select(
    "Customer_ID", "Customer_Name", "Segment", "City", "State", "Country", "Region"
).dropDuplicates()

# Join with dim_segment to add Segment_ID
dim_customer_with_segment_id = dim_customer.join(
    df_segment_dim, on="Segment", how="left"
)

# Optional: drop Segment column to normalize
dim_customer_final = dim_customer_with_segment_id.select(
    "Customer_ID", "Customer_Name", "Segment_ID", "City", "State", "Country", "Region"
)

# Write to Warehouse as table
dim_customer_final.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_customer')

# -------------------------------------------Dimension - Product-----------------------------------------------------
df_product = df_clean_rounded.dropDuplicates(["Product"])

#Add ProductID Column
df_product_dim = df_product.withColumn(
    "Product_ID", row_number().over(Window.orderBy("Product"))
)

df_product_dim = df_product_dim.select("Product_ID", "Product", "Product_Category")

# Write to Warehouse as table
df_product_dim.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_product')


# ---------------------------------------------Dimension - Order-------------------------------------------------------

# Extract order table
dim_order_raw = df_clean_rounded.select(
    "Order_ID", "Order_Date", "Shipping_Date", "Ship_Mode", "Order_Priority", "Aging"
).dropDuplicates()

# Join with dim_date twice (for Order_Date and Shipping_Date)
dim_order_with_order_date_id = dim_order_raw \
    .join(df_date_with_id.withColumnRenamed("BusinessDate", "Order_Date"), on="Order_Date", how="left") \
    .withColumnRenamed("Date_ID", "Order_Date_ID")

dim_order_with_both_dates = dim_order_with_order_date_id \
    .join(df_date_with_id.withColumnRenamed("BusinessDate", "Shipping_Date"), on="Shipping_Date", how="left") \
    .withColumnRenamed("Date_ID", "Shipping_Date_ID")

# Select final normalized columns
dim_order_final = dim_order_with_both_dates.select(
    "Order_ID",
    "Order_Date_ID",
    "Shipping_Date_ID",
    "Ship_Mode",
    "Order_Priority",
    "Aging"
)

dim_order_final.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_order')

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 10, Finished, Available, Finished)

##### Create Dimension Table from "Return"

In [9]:
from pyspark.sql import SparkSession

# Create the junk dimension DataFrame
dim_return_flag = spark.createDataFrame([
    (1, "Yes"),
    (0, "No")
], ["Return_ID", "Return_Description"])

# Write to Fabric Warehouse
dim_return_flag.write.mode('overwrite').synapsesql("warehouse_hv.sales.dim_return_flag")
display(dim_return_flag)

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 82e80355-53d4-4ccc-b130-24550135d042)

In [10]:
# Clean and map Return column
sales_return_cleaned = df_return_final.dropDuplicates() \
    .withColumn("Order_ID", trim(col("Order_ID"))) \
    .withColumn("Customer_Name", trim(col("Customer_Name"))) \
    .withColumn("Return_ID", when(trim(col("Return")) == "Yes", 1).otherwise(0)) \
    .withColumn("Sales_Amount", regexp_replace(col("Sales_Amount"), "[$]", "").cast("double"))

# Filter out rows with missing critical fields
sales_return_filtered = sales_return_cleaned.filter(
    col("Order_ID").isNotNull() & 
    col("Customer_Name").isNotNull() & 
    col("Sales_Amount").isNotNull()
)

# Map Return to 1/0
sales_return_mapped = sales_return_cleaned.withColumn(
    "Return_ID", when(col("Return") == "Yes", 1).otherwise(0)
)

dim_sales_return_final = sales_return_mapped.select(
    "Order_ID", "Customer_Name", "Sales_Amount", "Return_ID" 
)

# Write to Warehouse table
dim_sales_return_final.write.mode('overwrite').synapsesql('warehouse_hv.sales.dim_sales_return')
display(dim_sales_return_final.limit(5))

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 766ac8b9-2fe0-4482-85f1-a48cfee7eaf0)

##### Create a Fact Table

In [12]:
# Read dimension tables from Fabric Warehouse
dim_customer = spark.read.synapsesql("warehouse_hv.sales.dim_customer")
dim_product = spark.read.synapsesql("warehouse_hv.sales.dim_product")
dim_order = spark.read.synapsesql("warehouse_hv.sales.dim_order")
dim_sales_return = spark.read.synapsesql("warehouse_hv.sales.dim_sales_return")
dim_segment = spark.read.synapsesql("warehouse_hv.sales.dim_segment")
dim_date = spark.read.synapsesql("warehouse_hv.sales.dim_date")


# Add Product_ID
fact_joined = df_clean_rounded \
    .join(dim_product.select("Product", "Product_ID"), on="Product", how="left") \
    .join(dim_customer.select("Customer_ID", "Segment_ID"), on="Customer_ID", how="left") \
    .join(dim_order.select("Order_ID", "Order_Date_ID", "Shipping_Date_ID"), on="Order_ID", how="left") \
    .join(dim_sales_return.select("Order_ID", "Return_ID"), on="Order_ID", how="left")

fact_joined = fact_joined.withColumn("Return_ID", when(col("Return_ID").isNull(), 0).otherwise(col("Return_ID")))

# ---------------------------------- Final Fact Table ----------------------------------
fact_sales_final = fact_joined.select(
    "Order_ID",
    "Customer_ID",
    "Segment_ID",
    "Product_ID",
    "Order_Date_ID",
    "Shipping_Date_ID",
    "Sales",
    "Profit",
    "Quantity",
    "Discount",
    "Shipping_Cost",
    "Return_ID"
)

# Save fact table to Warehouse
fact_sales_final.write.mode('overwrite').synapsesql('warehouse_hv.sales.fact_sales')

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 14, Finished, Available, Finished)

##### Checking Fact table output:

In [14]:
display(fact_sales_final.limit(5))

StatementMeta(, 1aa0c786-07f3-4a04-9770-9d9b373d9d80, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4e6e6ae5-7dd4-4ba8-92a9-2c5b01fb50c1)